In [ ]:
# Step 1: Import Libraries & Load Dataset

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# Using CIFAR-10 subset as a simulated dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Use only "truck" and "automobile" as 2-class example
idx = np.where((y_train == 1) | (y_train == 9))
x_train, y_train = x_train[idx], y_train[idx]

idx = np.where((y_test == 1) | (y_test == 9))
x_test, y_test = x_test[idx], y_test[idx]

# Normalize
x_train, x_test = x_train/255.0, x_test/255.0


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
# Step 2: Build Lightweight Model

from tensorflow.keras import models, layers

# Convert labels to binary
y_train_bin = (y_train == 1).astype(int)
y_test_bin = (y_test == 1).astype(int)

model = models.Sequential([
    layers.Input(shape=(32, 3)),        # 32 timesteps, 3 features
    layers.Conv1D(16, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train_bin,
    epochs=5,
    batch_size=32,
    validation_split=0.2
)

test_loss, test_acc = model.evaluate(x_test, y_test_bin)
print("Test Accuracy:", test_acc)


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6200 - loss: 0.6568 - val_accuracy: 0.6470 - val_loss: 0.6371
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6571 - loss: 0.6283 - val_accuracy: 0.6420 - val_loss: 0.6455
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6459 - loss: 0.6332 - val_accuracy: 0.6530 - val_loss: 0.6356
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6463 - loss: 0.6322 - val_accuracy: 0.6545 - val_loss: 0.6347
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6535 - loss: 0.6269 - val_accuracy: 0.6585 - val_loss: 0.6342
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6840 - loss: 0.6165
Test Accuracy: 0.675000011920929


In [ ]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)


x_train shape: (10000, 32, 3)
y_train shape: (10000,)
x_test shape: (2000, 32, 3)
y_test shape: (2000,)


In [ ]:
# Step 3: Convert Model to TensorFlow Lite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("recycler_model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp30r2tx5a'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 3), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132228545701904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545702672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545700752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545701136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545703824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545703056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545704208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132228545704016: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Step 4: Test TFLite Model Inference

interpreter = tf.lite.Interpreter(model_path="recycler_model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

sample = np.expand_dims(x_test[0], axis=0).astype(np.float32)

interpreter.set_tensor(input_index, sample)
interpreter.invoke()

prediction = interpreter.get_tensor(output_index)
print("TFLite prediction:", prediction)


TFLite prediction: [[0.92903167]]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
